In [0]:
#conectar o blobstorage com o databricks
spark.conf.set(
  "fs.azure.account.key.cotaparlamentar.blob.core.windows.net",
  "gUyI/+FNpxY+DqGqYMn/0cvZSF1iXOx+oeho73jJO3QeSJ5OCFwcQ4dofmNQ1kt+1QYdq+OVKaf0j19aZJiISA==")

In [0]:
import csv
import gzip
import io
import json
import shutil
from urllib.parse import urlencode, urljoin
from urllib.request import Request, urlopen


class BrasilIO:

    base_url = "https://api.brasil.io/v1/"

    def __init__(self, user_agent=None, auth_token=None):
        """
        Caso queria fazer uma requisição na API, passe os parâmetros user_agent e auth_token.
        Para fazer somente o download do arquivo completo, não é necessário passar nenhum parâmetro.
        """
        self.__user_agent = user_agent
        self.__auth_token = auth_token

    def headers(self, api=True):
        if api:
            return {
                "User-Agent": f"{self.__user_agent}",
                "Authorization": f"Token {self.__auth_token}"
            }
        else:
            return {
                "User-Agent": "python-urllib/brasilio-client-0.1.0",
            }
            

    def api_request(self, path, query_string=None):
        url = urljoin(self.base_url, path)

        if query_string:
            url += "?" + query_string

        request = Request(url, headers=self.headers(api=True))

        response = urlopen(request)
        return json.load(response)
        
    def data(self, dataset_slug, table_name, filters=None):
        url = f"dataset/{dataset_slug}/{table_name}/data/"
        filters = filters or {}
        filters["page"] = 1

        finished = False
        while not finished:
            query_string = "&".join([f"{k}={v}" for k, v in filters.items()])
            response = self.api_request(url, query_string)
            next_page = response.get("next", None)
            for row in response["results"]:
                yield row
            filters = {}
            url = next_page
            finished = next_page is None

    def download(self, dataset, table_name):
        url = f"https://data.brasil.io/dataset/{dataset}/{table_name}.csv.gz"
        request = Request(url, headers=self.headers(api=False))
        response = urlopen(request)
        return response


if __name__ == "__main__":
    # Caso não tenha, cadastre-se no Brasil.io e gere seu Token
    # Para mais instruções: https://blog.brasil.io/2020/10/10/como-acessar-os-dados-do-brasil-io/
    user_agent = "fernando.oliveira"
    auth_token = "3c3a4f49112e84fb07ffd344d89f2f1455e81176"

    api = BrasilIO(user_agent, auth_token)

    dataset_slug = "gastos-deputados"
    table_name = "cota_parlamentar"

In [0]:
#chamada do metodo da classe
api = BrasilIO()
#usando a função download com os parametros do dataset
api.download('gastos-deputados','cota_parlamentar')

#download do arquivo
response = api.download(dataset_slug, table_name)
with open(f"{dataset_slug}_{table_name}.csv.gz", mode="wb") as fobj:
    fobj.write(response.read())

In [0]:
#descompactando arquivo
with gzip.open('gastos-deputados_cota_parlamentar.csv.gz', 'rb') as f_in:
    with open('gastos-deputados_cota_parlamentar.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [0]:
df = spark.read.options(header='true') \
          .csv("dbfs:/gastos-deputados_cota_parlamentar.csv")

In [0]:
dfl = df.na.fill("Não cadastrado")

In [0]:
dfl.coalesce(1).write.option("header","true").option('sep',',').format('com.databricks.spark.csv').save("wasbs://projectintegrat@cotaparlamentar.blob.core.windows.net/dataset/cota_parlamentar.csv")